# Modulo calculations

- <https://adventofcode.com/2021/day/21>

This is another modulo calculation problem, but there is no closed-form solution that I am aware of. So for part 1, we are just playing the game as stated, keeping a counter that goes up by 3 each round. The dice rolls are produced by [`itertools.cycle()`](https://docs.python.org/3/library/itertools.html#itertools.cycle) looping over the range from 1 through to 100 (inclusive).

In [1]:
from __future__ import annotations
import re
from collections import deque
from dataclasses import dataclass, replace
from itertools import count, cycle, islice
from typing import Callable, Iterator, Literal


Line: Callable[[str], re.Match] = re.compile(
    r"Player (?P<player>[12]) starting position: (?P<position>\d+)"
).fullmatch


class HundredDie:
    def __init__(self, per: int) -> None:
        self.values = cycle(range(1, 101))
        self.per = per

    def __iter__(self) -> Iterator[int]:
        values, per = self.values, self.per
        while True:
            yield sum(islice(values, per))


@dataclass(frozen=True)
class Player:
    player: Literal[1, 2]
    position: int
    score: int = 0

    @classmethod
    def from_line(cls, line: str) -> Player:
        group = Line(line).groupdict()
        return cls(int(group["player"]), int(group["position"]))

    def move(self, roll: int) -> None:
        position = (self.position + roll - 1) % 10 + 1
        return replace(self, position=position, score=self.score + position)


def play_game(p1: Player, p2: Player) -> tuple[int, int]:
    game = deque([p1, p2])
    per3 = HundredDie(3)
    for rolls, rolled in zip(count(3, 3), per3):
        player = game.popleft().move(rolled)
        if player.score >= 1000:
            return rolls, game[-1].score
        game.append(player)


test_players = [
    Player.from_line(line)
    for line in ("Player 1 starting position: 4", "Player 2 starting position: 8")
]
round, score = play_game(*test_players)
assert round * score == 739785


In [2]:
import aocd

players = [
    Player.from_line(line) for line in aocd.get_data(day=21, year=2021).splitlines()
]
round, score = play_game(*players)
print("Part 1:", round * score)


Part 1: 926610


# Part 2, a branching multiverse of possibilities

The quantum dice make this quite an interesting problem. The trick is to recognise that there can be multiple paths to the same outcomes here, and we only need to keep track of the number of universes for each set of player scores and count the number of winning states, *not* track each and every game or the state of dice rolls.

So, we count states (won or not), where a specific state is the score for each player, and produce new state counts from the current state counts while alternating between the two players. We can produce new state counts purely based on the probabilities of throwing a given dice sum.

At each round, the player can throw one of 27 different combinations of the dice, but there are only 7 different outcomes for those dice. Because each of the 27 possibilities results in a new universe, and we only want to count in how many universes a player wins the game, we only need to count are the _probabilities_ of each of those 27 possible outcomes. There is just one way for a throw of the 3 3-sided dice to total 9 (3 + 3 + 3) or 3 (1 + 1 + 1), but there are 7 different ways to throw a 6 (1 + 2 + 3, 1 + 3 + 2, 2 + 1 + 3, etc.). So if the outcome were to be 9, then there is still the same number of universes in which the new state (current player score + 9) can occur, but there are 7 times as many different universes in which the same player rolled dice and added 6 to their score. For the latter outcome, we multiply the count for the current state by 7 and use that to update the count for the new state.
 
Again, it should be noted that there can be multiple paths to a win for a given player; all universes where the opposing (not playing) player has score A and the current player is between 3 and 9 points away from score B, will produce a new state with scores A and B, and our job is to keep counts of the number of universes in which that specific state was reached. So for each round we keep a counter (multiset) per state, to use as the starting point for the next round.

Each time a state has reached a winning score for a player, that state is removed from consideration for the next round; we just keep a total for each player. Once we have run out of states that haven't yet got a winner, all we have to do is pick the larger of the two win counts.

In [4]:
from collections import Counter
from itertools import product
from typing import Final, NamedTuple


# with 3 3-sided dice, out of 27 possible rolls, how often will each total appear?
PROBS: Final[dict[int, int]] = Counter(
    sum(dice) for dice in product(range(1, 4), repeat=3)
)


class QuantumState(NamedTuple):
    """State of a game"""

    player1: Player
    player2: Player

    def update(self, roll: int, turn: int) -> QuantumState:
        if turn == 1:
            return self._replace(player1=self.player1.move(roll))
        return self._replace(player2=self.player2.move(roll))

    @property
    def winner(self) -> int | None:
        for player in self:
            if player.score >= 21:
                return player.player
        return None


def play_quantum_dice(p1: Player, p2: Player) -> int:
    states, wins = Counter([QuantumState(p1, p2)]), Counter()
    for turn in cycle((1, 2)):
        prev, states = states, Counter()
        for (state, universes), (roll, prob) in product(prev.items(), PROBS.items()):
            universes *= prob
            state = state.update(roll, turn)
            if winner := state.winner:
                wins[winner] += universes
                continue
            states[state] += universes
        if not states:
            break
    return max(wins.values())


assert play_quantum_dice(*test_players) == 444356092776315


In [5]:
print("Part 2:", play_quantum_dice(*players))


Part 2: 146854918035875
